In [29]:
import dgl
from dgl.nn.pytorch import  NNConv
import torch
import numpy as np
import torch.nn as nn
import pickle
from Newmodel import *

n=1536
drop_prob=0
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_heads=1
control_str='a'

def getmodel():
    g = pickle.load(open("UpdatedGraph_Neighbour10_Coarsen3", 'rb'))
    adjMatrix=g.adj(scipy_fmt='coo')
    aa=adjMatrix.todense()
    aa=torch.from_numpy(aa).float()

    def model(x):
        return torch.matmul(x[:,:,0,:],aa)
    return model,g

model,g=getmodel()
# model = STGCN_WAVE([1,1], 1, 1, n, g, drop_prob, device, num_heads, control_str).to(device)
model = MPNNGNN(g,node_in_feats=7, edge_in_feats=2, edge_hidden_feats=8, node_hidden_feats=9, node_out_feats=2,num_step_message_passing=9).to(device)


# x=torch.rand(1,1,1,n)
x=torch.rand(n,7)
y=torch.rand(1536*10,2)

x.requires_grad_()
y=model(x,y)
y[100,0].backward()
dx=x.grad
(dx!=0).sum()


tensor(2877)

In [24]:
class MPNNGNN(nn.Module):
    """MPNN.

    MPNN is introduced in `Neural Message Passing for Quantum Chemistry
    <https://arxiv.org/abs/1704.01212>`__.

    This class performs message passing in MPNN and returns the updated node representations.

    Parameters
    ----------
    node_in_feats : int
        Size for the input node features.
    node_out_feats : int
        Size for the output node representations. Default to 64.
    edge_in_feats : int
        Size for the input edge features. Default to 128.
    edge_hidden_feats : int
        Size for the hidden edge representations.
    num_step_message_passing : int
        Number of message passing steps. Default to 6.
    """
    def __init__(self, g, node_in_feats, edge_in_feats, node_hidden_feats=64,node_out_feats=64,
                 edge_hidden_feats=128, num_step_message_passing=6):
        super(MPNNGNN, self).__init__()

        self.project_node_feats = nn.Sequential(
            nn.Linear(node_in_feats, node_hidden_feats),
            nn.ReLU(),
            nn.Linear(node_hidden_feats, node_hidden_feats)
        )
        self.num_step_message_passing = num_step_message_passing
        edge_network = nn.Sequential(
            nn.Linear(edge_in_feats, edge_hidden_feats),
            nn.ReLU(),
            nn.Linear(edge_hidden_feats, node_hidden_feats * node_hidden_feats)
        )
        self.gnn_layer = NNConv(
            in_feats=node_hidden_feats,
            out_feats=node_hidden_feats,
            edge_func=edge_network,
            aggregator_type='sum'
        )
        self.gru = nn.GRU(node_hidden_feats, node_hidden_feats)

        self.decoder = nn.Sequential(nn.Linear( node_hidden_feats , node_hidden_feats),
                              nn.ReLU(),
                              nn.Linear( node_hidden_feats, node_out_feats)
                              )
        self.g=g

    def reset_parameters(self):
        """Reinitialize model parameters."""
        self.project_node_feats[0].reset_parameters()
        self.gnn_layer.reset_parameters()
        for layer in self.gnn_layer.edge_func:
            if isinstance(layer, nn.Linear):
                layer.reset_parameters()
        self.gru.reset_parameters()


    def forward(self, node_feats, edge_feats):
        """Performs message passing and updates node representations.

        Parameters
        ----------
        g : DGLGraph
            DGLGraph for a batch of graphs.
        node_feats : float32 tensor of shape (V, node_in_feats)
            Input node features. V for the number of nodes in the batch of graphs.
        edge_feats : float32 tensor of shape (E, edge_in_feats)
            Input edge features. E for the number of edges in the batch of graphs.

        Returns
        -------
        node_feats : float32 tensor of shape (V, node_out_feats)
            Output node representations.
        """
        node_feats = self.project_node_feats(node_feats) # (V, node_out_feats)
        hidden_feats = node_feats.unsqueeze(0)           # (1, V, node_out_feats)

        for _ in range(self.num_step_message_passing):
            # print(node_feats.shape)
            node_feats = F.relu(self.gnn_layer(self.g, node_feats, edge_feats))
            node_feats, hidden_feats = self.gru(node_feats.unsqueeze(0), hidden_feats)
            node_feats = node_feats.squeeze(0)

        return self.decoder(node_feats)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.MSELoss()
